# [BarlowTwins](https://arxiv.org/abs/2103.03230) on CIFAR-10

In [1]:
from os import makedirs

import torch
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision.models import resnet18

from models import BarlowTwins
#from optimizer import LARS
from trainer import SSL_Trainer
from utils import SSL_CIFAR10

# Define hyperparameters
data_root = '/home/space/datasets/'
#data_root = '/home/fcfschulz/Documents/workspace/data/Vision/'
save_root = './results/barlow_twins/'

dl_kwargs = {'batch_size': 512, 'shuffle': True, 'num_workers': 2}

# Define data
ssl_data = SSL_CIFAR10(data_root,'BYOL', dl_kwargs)

# general training params
train_params = {'save_root': save_root, 'num_epochs': 15, 'optimizer': SGD,
                'scheduler': CosineAnnealingLR, 'warmup_epochs': 10, 'iter_scheduler':True,
                'evaluate_at': [100,200,400,600], 'verbose':True}

# params of optimizer
## In Original Paper for Imagenet when using LARS Optimizer
#optim_params = {'lr':0.2 * dl_kwargs['batch_size']/256, 'weight_decay': 1.5e-6,
#                'exclude_bias_and_norm': True}

# from: https://github.com/IgorSusmelj/barlowtwins/blob/main/main.py
optim_params = {'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 5e-4} 

# params of scheduler
scheduler_params = {'T_max': (train_params['num_epochs']-train_params['warmup_epochs'])*len(ssl_data.train_dl)}
                    # 'eta_min': 1e-3} in orginal implementation

# Set parameters for fitting linear protocoler
eval_params  = {'lr':1e-2, 'num_epochs': 5, 'milestones': [12,20]}

# Get device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Print Device Type
if torch.cuda.is_available():
    print(f"Program running on {torch.cuda.get_device_name(device)}")
else:
    print("Program running on CPU")
    
# Create folder if it does not exists
makedirs(save_root, exist_ok=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Program running on NVIDIA A100-PCIE-40GB


# BarlowTwins

In [ ]:
# Define Model
resnet = resnet18(zero_init_residual=True)
barlow_twins = BarlowTwins(resnet, projector_hidden = (2048,2048,2048))

# Define Trainer
cifar10_trainer = SSL_Trainer(barlow_twins, ssl_data, device)

# Train
cifar10_trainer.train(**train_params, optim_params=optim_params,
                      scheduler_params=scheduler_params, eval_params=eval_params)

/home/fcfschulz/miniconda3/envs/pg_high/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0, Loss: 1430.2566043814434, Time epoch: 194.42377424240112
Epoch: 1, Loss: 1263.7786638712146, Time epoch: 204.28367161750793
Epoch: 2, Loss: 1100.4600811201271, Time epoch: 205.5447497367859
Epoch: 3, Loss: 994.5241560788498, Time epoch: 204.94365859031677
Epoch: 4, Loss: 919.3228734596488, Time epoch: 203.45735049247742
Epoch: 5, Loss: 847.7131926546392, Time epoch: 204.54951643943787
Epoch: 6, Loss: 807.5395086229462, Time epoch: 203.0283501148224
Epoch: 7, Loss: 766.2004772068299, Time epoch: 203.8757302761078
Epoch: 8, Loss: 735.5425635269007, Time epoch: 210.45079040527344
Epoch: 9, Loss: 720.8987256866141, Time epoch: 207.80512237548828
Epoch: 10, Loss: 686.5789776045023, Time epoch: 212.7353618144989
Epoch: 11, Loss: 661.3757097696521, Time epoch: 211.97152161598206
